In [4]:
# load all 35 csv files; join on LGA
import os, glob
import pandas as pd
import re
import numpy as np
base = "../../data/"
shapes = "raw/2016_LGA_SHAPE/LGA_2016_AUST"
path ='raw/2016_demographics/2016 Census GCP Local Government Areas for NSW/' 
allFiles = glob.glob(base+path + "*.csv")
dfs = []
for filename in allFiles:
    dfs.append((pd.read_csv(filename,index_col='LGA_CODE_2016', header=0), re.search("2016Census_(.*)_NSW_LGA", filename).group(1)))


In [64]:
dfs[0]

(               OPDs_Separate_house_Dwellings  OPDs_Separate_house_Persons  \
 LGA_CODE_2016                                                               
 LGA10050                               15634                        39601   
 LGA10130                                8689                        22010   
 LGA10250                               11223                        29084   
 LGA10300                                 734                         1907   
 LGA10470                               12166                        32367   
 LGA10550                               10934                        25770   
 LGA10600                                4466                        10684   
 LGA10650                                2904                         6728   
 LGA10750                               78774                       257809   
 LGA10800                                2066                         4970   
 LGA10850                                2507                   

In [65]:
import shapefile
sf = shapefile.Reader(base+shapes)

df = pd.DataFrame(sf.records())
nswdf = df[df[4] =="New South Wales"]
# drop "No usual address" - not a useful LGA
nswdf = nswdf[nswdf[0] != "LGA19499"]
nswdf.columns = ["code", "codenum", "name", "unknown", "state", "area"]
nswdf["clean_name"] = nswdf["name"].str.extract('([^\\(]*)', expand=False).str.strip()

amalgamations = {'Botany Bay': 'Bayside',
                 'Rockdale'  : 'Bayside',
                 'Gundagai'  : 'Cootamundra-Gundagai',
                 'Western Plains Regional' : 'Dubbo Regional',
                 'Unincorporated NSW' : 'Unincorporated Far West'
                }

nswdf["clean_name"]= nswdf["clean_name"].apply(lambda x: amalgamations.get(x) if x in amalgamations else x)

nswdf

,code,codenum,name,unknown,state,area,clean_name
0,LGA10050,10050,Albury (C),1,New South Wales,305.9459,Albury
1,LGA10130,10130,Armidale Regional (A),1,New South Wales,8620.6990,Armidale Regional
2,LGA10250,10250,Ballina (A),1,New South Wales,484.9389,Ballina
3,LGA10300,10300,Balranald (A),1,New South Wales,21690.6753,Balranald
4,LGA10470,10470,Bathurst Regional (A),1,New South Wales,3817.8646,Bathurst Regional
5,LGA10550,10550,Bega Valley (A),1,New South Wales,6278.8811,Bega Valley
6,LGA10600,10600,Bellingen (A),1,New South Wales,1600.4337,Bellingen
7,LGA10650,10650,Berrigan (A),1,New South Wales,2065.7759,Berrigan
8,LGA10750,10750,Blacktown (C),1,New South Wales,240.0487,Blacktown
9,LGA10800,10800,Bland (A),1,New South Wales,8557.6535,Bland


In [66]:
outputdf = dfs[0][0]
filename = dfs[0][1]
outputdf.columns = [x+"_"+filename for x in outputdf.columns]

for df, filename in dfs[1:]:
    df.columns = [x+"_"+filename for x in df.columns]
    outputdf = outputdf.join(df)

namedf = outputdf.merge(nswdf[['code', 'area', 'clean_name']], left_index=True, right_on='code')
namedf = namedf.drop('code', axis=1)

#now sum amalgamations

namedf = namedf[[x for x in namedf.columns if not "median" in x.lower() and not "average" in x.lower() and not "perc" in x.lower() ]]
namedf = namedf.groupby("clean_name").agg("sum").reset_index()
namedf= namedf.rename(columns={"clean_name":"lga"})
namedf.to_csv("2016_demographics.csv")

In [5]:
import pandas as pd
namedf = pd.read_csv(base+"/processed/2016_demographics.csv")

In [8]:
namedf.lga.unique()

array(['Albury', 'Armidale Regional', 'Ballina', 'Balranald',
       'Bathurst Regional', 'Bayside', 'Bega Valley', 'Bellingen',
       'Berrigan', 'Blacktown', 'Bland', 'Blayney', 'Blue Mountains',
       'Bogan', 'Bourke', 'Brewarrina', 'Broken Hill', 'Burwood', 'Byron',
       'Cabonne', 'Camden', 'Campbelltown', 'Canada Bay',
       'Canterbury-Bankstown', 'Carrathool', 'Central Coast',
       'Central Darling', 'Cessnock', 'Clarence Valley', 'Cobar',
       'Coffs Harbour', 'Coolamon', 'Coonamble', 'Cootamundra-Gundagai',
       'Cowra', 'Cumberland', 'Dubbo Regional', 'Dungog', 'Edward River',
       'Eurobodalla', 'Fairfield', 'Federation', 'Forbes',
       'Georges River', 'Gilgandra', 'Glen Innes Severn',
       'Goulburn Mulwaree', 'Greater Hume Shire', 'Griffith', 'Gunnedah',
       'Gwydir', 'Hawkesbury', 'Hay', 'Hilltops', 'Hornsby',
       'Hunters Hill', 'Inner West', 'Inverell', 'Junee', 'Kempsey',
       'Kiama', 'Ku-ring-gai', 'Kyogle', 'Lachlan', 'Lake Macquarie',
  

In [68]:
[x for x in namedf.columns if 'aver' in x.lower()]

[]